# Caso Práctico (tips) - ENGIE
*Jose Enrique Zafra Mena*

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl
import json 

## Carga de los datos

In [36]:
facturas_df = pd.read_excel('datos/datos-tips/AGIkey_facturas.xlsx')
mapping_df = pd.read_excel('datos/Mapping.xlsx')
IAC_df = pd.read_excel('datos/datos-tips/MEDG2_IAC_GETRA_ABRIL_2024_06_18_13_34.xlsx')

In [37]:
facturas_df.head(1000)

,NumeroFactura,Emisor,RazonSocialEmisor,DepEmisor,Receptor,RazonSocialReceptor,DepReceptor,FechaFactura,Importe,Moneda,...,FechaRegistro,Destino,Contrato,Origen,ServicioFacturado,¿Verificada por BO?,Observaciones,Importe (<0 a pagar EEE-Enagas),mes,año
0,1400180000,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"4,56",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
1,1400180001,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"15794,20",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
2,1400180002,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"16380,50",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
3,1400180003,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"1930,22",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
4,1400180004,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,2024-02-01 00:00:00,"43388,45",EUR,...,2024-02-01 00:00:00,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,1400180104,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,28/05/2024,"44751,03",NaN,...,28/05/2024,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
936,1400180105,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,28/05/2024,"11928,65",NaN,...,28/05/2024,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
937,1400180106,ESB82508441,ENGIE ESPAÑA SLU,20.0,ESA86484292,"ENAGAS GTS, SAU",NaN,28/05/2024,"11198,08",NaN,...,28/05/2024,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN
938,2024069931,ESA86484292,"ENAGAS GTS, SAU",20.0,ESB82508441,ENGIE ESPAÑA SLU,NaN,28/05/2024,"16141,36",NaN,...,28/05/2024,eMail,344991,PVB,LiquidaciÃ³n Desbalance Provisional,NaN,NaN,NaN,NaN,NaN


## Filtrado de datos relevantes

### Facturas
Solo nos interesan ciertas columnas: Factura, Importe, Fecha Factura.

In [38]:
facturas_df = facturas_df.drop_duplicates()

facturas_df = facturas_df[['NumeroFactura', 'Importe', 'FechaFactura', 'Origen','ServicioFacturado']]

# solo queremos algunos valores de Origen: C.I.Almería y Entrada PVB
facturas_filtrado_df = facturas_df[(facturas_df['Origen'].isin(['C.I. Almería'])) & (facturas_df['ServicioFacturado'].isin(['Entrada PVB']))]


facturas_filtrado_df.head(10)

,NumeroFactura,Importe,FechaFactura,Origen,ServicioFacturado
20,2324001054,"537278,06",2024-03-01 00:00:00,C.I. Almería,Entrada PVB
159,2324002755,"514862,14",2024-05-02 00:00:00,C.I. Almería,Entrada PVB
376,2324004231,"503763,47",2024-06-03 00:00:00,C.I. Almería,Entrada PVB
570,2324005598,"536445,62",2024-04-04 00:00:00,C.I. Almería,Entrada PVB
771,2324007434,"525023,53",2024-07-05 00:00:00,C.I. Almería,Entrada PVB


### Mapping

In [39]:
mapping_df = mapping_df.drop_duplicates()
print(mapping_df.shape)

# solo nos interesan algunos valores: Commodity == IAC
mapping_filtrado_df = mapping_df[(mapping_df['Commodity'].isin(['IAC']))]
mapping_filtrado_df.head(200)

(29, 5)


,Origen,Servicio facturado,Portfolio,Commodity,DealType
0,C.I. Almería,Entrada PVB,MEDG2,IAC,GETRA


### Deals 
Solo queremos las columnas: Deal, Cantidad, Start-DeliveryDate, End-DeliveryDate, Procedencia (SUGST_AOC_STOK), Subflow 1, Date 1, Subcantidad 1, Subflow 2, Date 2, Subcantidad 2.

En la columna flows vemos que hay datos en json. Tratamos esa columna para sacar los datos a un formato mejor.

In [47]:
IAC_df = IAC_df.drop_duplicates()

# nos quedamos con las columnas que queremos por ahora
IAC_filtrado_df = IAC_df[['Id','Flows','StartDeliveryDate', 'EndDeliveryDate']]
IAC_filtrado_df = IAC_filtrado_df.rename(columns={'Id':'Id Deal'})


# definimos una función para tratar los json (la tendremos que usar varias veces)
## se usan dobles comillas, null y true
def clean_json(str):
    str = str.replace("'", '"')
    str = str.replace("None", "null")
    str = str.replace("True", "true")
    str = str.replace("False", "false")
    return str

# la usamos sobre la columna de flows,
flows_cleaned = clean_json(IAC_filtrado_df['Flows'])

# ahora creamos otra función para extraer los datos de json a diccionarios de python,
def extract_json(flows):
    data = {
        'Currency': [],
        'Amount': [],
        'Id': [],
        'Date':[],
        'FlowType': [],
        'PayReceive': []
    }
    for item in flows:
        for key in data.keys():
            data[key].append(item.get(key))
    return data

datos_json = []

for data in flows_cleaned.apply(extract_json):
    i = pd.DataFrame(data)
    datos_json.append(i)


datos_json_final = pd.concat(datos_json)

datos_json_final.head()

AttributeError: 'str' object has no attribute 'get'

## Merge entre Facturas y Deals


## Conciliación
**Se hace según *Origen* + *Servicio Facturado***

Mapping y Facturas comparten *Origen* y *Servicio Facturado*

Vemos la cantidad según Facturas, la cantidad según Deals, y hacemos la diferencia